In [1]:
import dgl
from scipy import spatial
import numpy as np
import pandas as pd

df = pd.read_excel('./dataset/Beauty/data.xlsx')
# merged_df = pd.read_excel('./dataset/Haircare/merged_df.xlsx')
# attributes = pd.read_excel('./dataset/Haircare/attributes.xlsx')
g = dgl.load_graphs('./dataset/Beauty/beauty_data_except_perfume.dgl')[0][0]

saved_npz = np.load('./multisage/' + 'Beauty_h_items.npz')
h_item = saved_npz['item_vectors']
tree = spatial.KDTree(h_item.tolist())

In [8]:
entities = df['ID'].astype('category')
m_entities = entities.cat.reorder_categories(df['ID'].values)

index_id_to_ID = {}
ID_to_index_id = {}
for idx, ID in enumerate(m_entities):
    index_id_to_ID[idx] = ID
    ID_to_index_id[ID] = idx
    
h_query = h_item[ID_to_index_id[18900649628]]
index_ids = tree.query(h_query, 50)[1]
IDs = [index_id_to_ID[idx] for idx in index_ids]

for mid in IDs:
    print(df[df['ID']==mid]['상품명'].values)

['큐어 워터 스플래쉬 쿨링 선스틱 23g(SPF50+)']
['밸런스 피에이치 마스크 50ml']
['알로에베라 젤 200ml']
['메디 셀 마스크 25g']
['로얄 비타프로폴리스 마스크 30ml']
['엑스트랙션 인 어 프랙션 120ml']
['아쿠아 블루 로즈 모이스쳐 모델링 팩 1제 1000g + 2제 100g']
['아보카도 에센스 마스크팩 23g']
['콜라겐 마스크팩 26g']
['로이스 워터 솔루블 마사지 오일 라벤더 1000ml']
['초공진 금설 진 마스크 30g']
['라센스 로에 큐어 워터 스플래쉬 쿨링선스틱 23g(SPF50+)']
['프리미엄 알엑스 셀 트리트먼트 마스크 23ml']
['플랜트 셀 2스텝 마스크 리프팅필 35g']
['리얼 핏 세컨드 스킨 탄력 레드팩 28g']
['큐어 워터 스플래쉬 쿨링 선스틱 23g(신형,SPF50+)']
['오리지널 콜라겐 테라피 포헤드']
['악센 SOS 슬리핑 마스크 50ml']
['히아루로닉 글래시어 무스 30g']
['데일리 톤 업 선크림 50ml(SPF50+)']
['홍삼 프리미엄 바이탈 마스크팩 25g']
['G R 필 오프 젤 마스크 500g']
['마데카 더마 쉴드 쿨링 선스틱 22g(SPF50+)']
['인텐시브 리바이탈라이징 마스크 75ml']
['마데세라 슬리핑 마스크 50ml']
['석류 내추럴 에센스 마스크']
['프로폴리스 에너지 앰플 마스크 20g']
['밀크 앤 필 마스크 60g']
['메이크 마이 데이 마스크팩 7g']
['세이브더스킨 아토 패치 3.5g']
['검정콩 유기농 마스크 팩']
['초공진 영안 진 마스크 18g']
['더마 사이언스 조조바 앤 허니 마사지 젤 300g']
['글로우 디 시카 랩핑 마스크 17g']
['한방 분말 해초팩 260g']
['쎌루라 하이드라리프트 퍼밍 마스크 50ml']
['찹쌀 쫀쫀 팩 60ml']
['마데카 더마쉴드 쿨링 선스틱 22g(SPF50+)']
['아로마 마사지 크림 1000ml']
['딥 클렌징 마스크 250ml

In [9]:
import torch
import pickle
import sys

sys.path.append('C:/Users/kimhw/Desktop/데이터 청년 캠퍼스/Github/ESG-Recommender-System-by-MultiSAGE/multisage')
from model import MultiSAGEModel
from sampler import ItemToItemBatchSampler, NeighborSampler, PinSAGECollator

load_dict = torch.load('./multisage/Beauty_MultiSAGE_weights.pth')

model = MultiSAGEModel(g, '상품', '특징', 512, 2, 3)
model.load_state_dict(load_dict)

batch_sampler = ItemToItemBatchSampler(g, '특징', '상품', 512)
neighbor_sampler = NeighborSampler(
    g, '특징', '상품', 2, 0.5, 10, 5, 2)
collator = PinSAGECollator(neighbor_sampler, g, '상품', '특징')

In [11]:
index_id = ID_to_index_id[18900649628]
with torch.no_grad():
    blocks, context_blocks = collator.collate_point(index_id=index_id)
    context_batch = model.get_representation(blocks, context_blocks, context_id=0)
    
tree.query(context_batch.numpy()[0], 10)

index_ids = tree.query(context_batch.numpy()[0], 50)[1]
IDs = [index_id_to_ID[idx] for idx in index_ids]
for mid in IDs:
    print(df[df['ID']==mid]['상품명'].values)

['큐어 워터 스플래쉬 쿨링 선스틱 23g(SPF50+)']
['밸런스 피에이치 마스크 50ml']
['알로에베라 젤 200ml']
['메디 셀 마스크 25g']
['로얄 비타프로폴리스 마스크 30ml']
['엑스트랙션 인 어 프랙션 120ml']
['아쿠아 블루 로즈 모이스쳐 모델링 팩 1제 1000g + 2제 100g']
['아보카도 에센스 마스크팩 23g']
['로이스 워터 솔루블 마사지 오일 라벤더 1000ml']
['콜라겐 마스크팩 26g']
['라센스 로에 큐어 워터 스플래쉬 쿨링선스틱 23g(SPF50+)']
['초공진 금설 진 마스크 30g']
['플랜트 셀 2스텝 마스크 리프팅필 35g']
['프리미엄 알엑스 셀 트리트먼트 마스크 23ml']
['리얼 핏 세컨드 스킨 탄력 레드팩 28g']
['오리지널 콜라겐 테라피 포헤드']
['큐어 워터 스플래쉬 쿨링 선스틱 23g(신형,SPF50+)']
['악센 SOS 슬리핑 마스크 50ml']
['히아루로닉 글래시어 무스 30g']
['데일리 톤 업 선크림 50ml(SPF50+)']
['G R 필 오프 젤 마스크 500g']
['인텐시브 리바이탈라이징 마스크 75ml']
['홍삼 프리미엄 바이탈 마스크팩 25g']
['마데카 더마 쉴드 쿨링 선스틱 22g(SPF50+)']
['밀크 앤 필 마스크 60g']
['마데세라 슬리핑 마스크 50ml']
['메이크 마이 데이 마스크팩 7g']
['세이브더스킨 아토 패치 3.5g']
['석류 내추럴 에센스 마스크']
['검정콩 유기농 마스크 팩']
['프로폴리스 에너지 앰플 마스크 20g']
['초공진 영안 진 마스크 18g']
['더마 사이언스 조조바 앤 허니 마사지 젤 300g']
['쎌루라 하이드라리프트 퍼밍 마스크 50ml']
['한방 분말 해초팩 260g']
['찹쌀 쫀쫀 팩 60ml']
['마데카 더마쉴드 쿨링 선스틱 22g(SPF50+)']
['타임 리버스 머드 크림 마스크 150g']
['악센 포어 컨트롤 마스크 200ml(복합, 지성)']
['아로마